In [24]:
# This is necessary to recognize the modules
import os
import sys
from decimal import Decimal

root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)

In [25]:
from hummingbot.strategy_v2.backtesting.controllers_backtesting.market_making_backtesting import MarketMakingBacktesting
from controllers.market_making.avellaneda_stoikov_backtesting_v2 import AvellanedaStoikovBacktestingV2ControllerConfig
import datetime

In [3]:
backtesting_engine = MarketMakingBacktesting()
config = AvellanedaStoikovBacktestingV2ControllerConfig(
    connector_name="binance_perpetual",
    trading_pair="ETH-USDT",
    total_amount_quote=100.0,
    executor_refresh_time=300,
    target_base_quote_ratio=0.5,
    inventory_risk_aversion=0.5,
    gamma=2.0,
    kappa=7.25,
    candles_connector="binance",
    candles_trading_pair="ETH-USDT",
    interval="30m",
    max_records=1000,
    natr_length=30,
    stop_loss=0.03,
    take_profit=0.02,
    time_limit=2700,
)


In [4]:
start_time = datetime.datetime(2024, 9, 10).timestamp()
end_time = datetime.datetime(2024, 9, 11).timestamp()

In [5]:
backtesting_results = await backtesting_engine.run_backtesting(
    controller_config=config, trade_cost=0.0006,
    start=int(start_time), end=int(end_time), backtesting_resolution="1m")
df = backtesting_results["processed_data"]["features"]
executors = backtesting_results["executors"]
results = backtesting_results["results"]

2024-09-13 21:16:47,529 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x146ba9b10>
2024-09-13 21:16:47,530 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x146b4f100>, 484321.094751041)]']
connector: <aiohttp.connector.TCPConnector object at 0x146ba9570>
2024-09-13 21:16:48,951 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x146baa740>
2024-09-13 21:16:48,953 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x146b4f460>, 484322.533974458)]']
connector: <aiohttp.connector.TCPConnector object at 0x146baa8f0>


In [19]:
# Print all column names of df
print(backtesting_results["processed_data"].keys())

dict_keys(['reference_price', 'reservation_price', 'optimal_spread', 'inventory', 'base_balance', 'quote_balance', 'optimal_one_side_spread', 'buy_spread_pct', 'sell_spread_pct', 'features', 'spread_multiplier'])


In [23]:
quote_balance = backtesting_results["processed_data"]["quote_balance"]
base_balance = backtesting_results["processed_data"]["base_balance"]

inventory = backtesting_results["processed_data"]["inventory"]
inventory

Decimal('0')

In [9]:

from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
import plotly.graph_objects as go

from hummingbot.connector.connector_base import TradeType



def get_bt_candlestick_trace(df):
    df.index = pd.to_datetime(df.timestamp, unit='s')
    return go.Scatter(x=df.index,
                      y=df['close'],
                      mode='lines',
                      line=dict(color="blue"),
                      )

def get_pnl_trace(executors):
    pnl = [e.net_pnl_quote for e in executors]
    cum_pnl = np.cumsum(pnl)
    return go.Scatter(
        x=pd.to_datetime([e.close_timestamp for e in executors], unit="s"),
        y=cum_pnl,
        mode='lines',
        line=dict(color='gold', width=2, dash="dash"),
        name='Cumulative PNL'
    )

def get_default_layout(title=None, height=800, width=1800):
    layout = {
        "template": "plotly_dark",
        "plot_bgcolor": 'rgba(0, 0, 0, 0)',  # Transparent background
        "paper_bgcolor": 'rgba(0, 0, 0, 0.1)',  # Lighter shade for the paper
        "font": {"color": 'white', "size": 12},  # Consistent font color and size
        "height": height,
        "width": width,
        "margin": {"l": 20, "r": 20, "t": 50, "b": 20},
        "xaxis_rangeslider_visible": False,
        "hovermode": "x unified",
        "showlegend": False,
    }
    if title:
        layout["title"] = title
    return layout

def add_executors_trace(fig, executors, row, col):
    for executor in executors:
        entry_time = pd.to_datetime(executor.timestamp, unit='s')
        entry_price = executor.custom_info["current_position_average_price"]
        exit_time = pd.to_datetime(executor.close_timestamp, unit='s')
        exit_price = executor.custom_info["close_price"]
        name = "Buy Executor" if executor.config.side == TradeType.BUY else "Sell Executor"

        if executor.filled_amount_quote == 0:
            fig.add_trace(go.Scatter(x=[entry_time, exit_time], y=[entry_price, entry_price], mode='lines',
                                     line=dict(color='grey', width=2, dash="dash"), name=name), row=row, col=col)
        else:
            if executor.net_pnl_quote > Decimal(0):
                fig.add_trace(go.Scatter(x=[entry_time, exit_time], y=[entry_price, exit_price], mode='lines',
                                         line=dict(color='green', width=3), name=name), row=row, col=col)
            else:
                fig.add_trace(go.Scatter(x=[entry_time, exit_time], y=[entry_price, exit_price], mode='lines',
                                         line=dict(color='red', width=3), name=name), row=row, col=col)
                
        # print(f"Entry_time: {entry_time} Exit time: {exit_time}, Exit price: {exit_price}")
    return fig


def create_backtesting_figure(df, executors, config):
    # Create subplots
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                        vertical_spacing=0.02, subplot_titles=('Candlestick', 'PNL Quote'),
                        row_heights=[0.7, 0.3])

    # Add candlestick trace
    fig.add_trace(get_bt_candlestick_trace(df), row=1, col=1)

    # Add executors trace
    fig = add_executors_trace(fig, executors, row=1, col=1)

    # Add PNL trace
    fig.add_trace(get_pnl_trace(executors), row=2, col=1)

    # Apply the theme layout
    layout_settings = get_default_layout(f"Trading Pair: {config['trading_pair']}")
    layout_settings["showlegend"] = False
    fig.update_layout(**layout_settings)

    # Update axis properties
    fig.update_xaxes(rangeslider_visible=False, row=1, col=1)
    fig.update_xaxes(row=2, col=1)
    fig.update_yaxes(title_text="Price", row=1, col=1)
    fig.update_yaxes(title_text="PNL", row=2, col=1)
    return fig

fig = create_backtesting_figure(
        df=df,
        executors=executors,
        config=config.dict())
cols_to_show = ["net_pnl_quote", "total_volume", "close_types", "sharpe_ratio", "profit_factor", "max_drawdown_usd"]
print(results)
fig.show()

{'net_pnl': -3.650162636974934, 'net_pnl_quote': -3650.162636974934, 'total_executors': 238, 'total_executors_with_position': 32, 'total_volume': 13435896.855344737, 'total_long': 0, 'total_short': 32, 'close_types': {'EARLY_STOP': 205, 'TIME_LIMIT': 33}, 'accuracy_long': 0.0, 'accuracy_short': 0.5, 'total_positions': 32, 'accuracy': 0.5, 'max_drawdown_usd': -6901.402326363093, 'max_drawdown_pct': -22.464221132925797, 'sharpe_ratio': -1.8423413186469333, 'profit_factor': 0.7565651662666528, 'win_signals': 16, 'loss_signals': 16}
